In [14]:
from langgraph.graph import StateGraph
from dotenv import  load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import TypedDict, Annotated
from pydantic import BaseModel, Field
import operator

In [15]:
load_dotenv()

True

In [16]:
model = ChatGoogleGenerativeAI(model = "gemini-2.5-flash")

In [8]:
class Evaluation(BaseModel):
    feedback : str = Field(description="detailed feed back of the essay")
    score : float = Field(description="score out of 10",ge=0,le=10)


struct_model = model.with_structured_output(Evaluation)

In [17]:
res = struct_model.invoke("India is a vast and diverse nation with a rich civilizational heritage that spans thousands of years. Known for its cultural plurality, it is home to numerous religions, languages, and traditions that coexist under the framework of the world’s largest democracy. Since gaining independence in 1947, India has progressed significantly in science, technology, and economic development, while still addressing challenges such as inequality and infrastructure gaps. From the Himalayas to the Indian Ocean, India’s geography, festivals, cuisine, and arts reflect a deep sense of identity and continuity, making it a unique blend of tradition and modernity.")
print(res)

feedback="This essay provides a concise yet comprehensive overview of India. It effectively touches upon key aspects such as its civilizational heritage, cultural diversity, democratic framework, post-independence progress in science, technology, and economy, and acknowledges persistent challenges like inequality. The concluding sentence beautifully encapsulates India's unique blend of tradition and modernity through its geography, festivals, cuisine, and arts. The language is clear, the structure is logical, and the flow is smooth, making it an engaging read. To make it even stronger, one could consider adding a specific example for the advancements or challenges mentioned, though for a general overview, it is excellent as is." score=9.0


In [18]:
res.score

9.0

In [22]:
class UPSCstate(TypedDict):
    essay: str 
    language_feedback : str 
    analysis_feedback : str 
    clarity_feedback : str 
    overall_feedback : str 
    individual_scores : Annotated[list[int], operator.add]
    avg_score : float

In [ ]:
def evaluate_language(state : UPSCstate):
    prompt = f"Evaluate the language in the essay and give me short feed back and score out of 10 essay : {state['essay']}"

    result = struct_model.invoke(prompt)

    return {'language_feedback':result.feedback, 'individual_scores': [result.score]}

def evaluate_analysis(state : UPSCstate):
    prompt = f"Evaluate the analytical skill in essay and give me short feed back and score out of 10 essay : {state['essay']}"

    result = struct_model.invoke(prompt)

    return {'analysis_feedback':result.feedback, 'individual_scores': [result.score]}

def evaluate_thought(state : UPSCstate):
    prompt = f"Evaluate the clarity and creative skill in essay and give me short feed back and score out of 10 essay : {state['essay']}"

    result = struct_model.invoke(prompt)

    return {'clarity_feedback':result.feedback, 'individual_scores': [result.score]}

In [ ]:
graph = StateGraph(UPSCstate)

graph.add_node('evaluate_language' , evaluate_language)
graph.add_node('evaluate_analysis',evaluate_analysis)
graph.add_node('evaluate_thought',evaluate_thought)
graph.add_node('final_evaluation',final_evaluation)
